In [ ]:
from tensorflow.keras.layers import Input,Dense ,Dropout
import numpy as np  
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input,Dense,Dropout,LeakyReLU 
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
import gc
from sklearn.impute import SimpleImputer
import scipy.stats as spstats

In [ ]:
path="/kaggle/input/predict-volcanic-eruptions-ingv-oe/"
test=pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
train=pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')



In [ ]:
sample_train=pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/1004561781.csv')
sample_train

In [ ]:
sample_train.isnull().sum()

In [ ]:
fig , axs = plt.subplots(nrows=2,ncols=5)
fig.set_size_inches(50,7)
fig.subplots_adjust(hspace=1)

for col,ax in zip(sample_train.columns, axs.flatten()):
    ax.plot(range(len(sample_train[col])),sample_train[col])
    ax.set_title(col)

# Feature Extraction

In [ ]:
def extract(dir_="train"):
    
    if dir_=="train":
        data=pd.read_csv("/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv")
    elif dir_=="test":
        data=pd.read_csv("/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")
    else:
        raise KeyError("noob")
        
    i=0
    
    for seg_id in data['segment_id']:
        
        f = pd.read_csv(f"/kaggle/input/predict-volcanic-eruptions-ingv-oe/{dir_}/{seg_id}.csv")

        # Fill NaN
        f.interpolate(axis=0,inplace=True)


        for sensor in f.columns:

            data.loc[i:i+1,f'{sensor}_mean']=f[f'{sensor}'].mean(axis=0)
            data.loc[i,f'{sensor}_std']=f[f'{sensor}'].std(axis=0)
            data.loc[i,f'{sensor}_max']=f[f'{sensor}'].max(axis=0)
            data.loc[i,f'{sensor}_min']=f[f'{sensor}'].min(axis=0)
            data.loc[i,f'{sensor}_mad']=f[f'{sensor}'].mad(axis=0)
            data.loc[i,f'{sensor}_skew']=f[f'{sensor}'].skew(axis=0)
            data.loc[i,f'{sensor}_kurt']=f[f'{sensor}'].kurt(axis=0)
            data.loc[i,f'{sensor}_var']=f[f'{sensor}'].var(axis=0)
            data.loc[i,f'{sensor}_median']=f[f'{sensor}'].median(axis=0)

            data.loc[i,f'{sensor}_nunique']=f[f'{sensor}'].nunique()

            data.loc[i,f'{sensor}_q005'] = f[f'{sensor}'].quantile(0.05)
            data.loc[i,f'{sensor}_q010'] = f[f'{sensor}'].quantile(0.1 )
            data.loc[i,f'{sensor}_q030'] = f[f'{sensor}'].quantile(0.3 )
            data.loc[i,f'{sensor}_q070'] = f[f'{sensor}'].quantile(0.7 )
            data.loc[i,f'{sensor}_q090'] = f[f'{sensor}'].quantile(0.9 )
            data.loc[i,f'{sensor}_q095'] = f[f'{sensor}'].quantile(0.95)
            data.loc[i,f'{sensor}_q999'] = f[f'{sensor}'].quantile(0.999)
            data.loc[i,f'{sensor}_q87'] = f[f'{sensor}'].quantile(0.8)
            data.loc[i,f'{sensor}_q13'] = f[f'{sensor}'].quantile(0.13)  
            data.loc[i,f'{sensor}_q01'] = f[f'{sensor}'].quantile(0.01)
            data.loc[i,f'{sensor}_q001'] = f[f'{sensor}'].quantile(0.001)

            #f[f'{sensor}_pctchange'e(f[f'{sensor}]).pct_change(period=)
            #f[f'{sensor}_pctchange']=pd.DataFrame(f[f'{sensor}']).pct_change(period=)
            #f[f'{sensor}_pctchange']=pd.DataFrame(f[f'{sensor}']).pct_change(period=)
            #f[f'{sensor}_pctchange']=pd.DataFrame(f[f'{sensor}']).pct_change(period=)
           # data.iloc[i,[f'{sensor}_pctchange']=pd.DataFrame(f[f'{sensor}']).pct_change(period=)

            x_abs=np.abs(f[f'{sensor}'])
            data.loc[i,f'{sensor}_q999_abs'] = np.quantile(x_abs, 0.999)
            data.loc[i,f'{sensor}_q99_abs']  = np.quantile(x_abs, 0.99)
            data.loc[i,f'{sensor}_q95_abs']  = np.quantile(x_abs, 0.95)
            data.loc[i,f'{sensor}_q87_abs']  = np.quantile(x_abs, 0.87)
            data.loc[i,f'{sensor}_q13_abs']  = np.quantile(x_abs, 0.13)
            data.loc[i,f'{sensor}_q05_abs']  = np.quantile(x_abs, 0.05)
            data.loc[i,f'{sensor}_q01_abs']  = np.quantile(x_abs, 0.01)
            data.loc[i,f'{sensor}_q001_abs'] = np.quantile(x_abs, 0.001)

            z = np.fft.fft(f[f'{sensor}'].fillna(0))
            fft_real = np.real(z)
            fft_imag = np.imag(z)
            data.loc[i,f'{sensor}_fft_real_mean'] = fft_real.mean()
            data.loc[i,f'{sensor}_fft_real_std']  = fft_real.std()
            data.loc[i,f'{sensor}_fft_real_max']= fft_real.max()
            data.loc[i,f'{sensor}_fft_real_min']= fft_real.min()
            data.loc[i,f'{sensor}_fft_real_median'] = np.median(fft_real)
            data.loc[i,f'{sensor}_fft_real_skew'] = spstats.skew(fft_real)
            data.loc[i,f'{sensor}_fft_real_kurtosis']= spstats.kurtosis(fft_real)

            data.loc[i,f'{sensor}_fft_imag_mean'] = fft_imag.mean()
            data.loc[i,f'{sensor}_fft_imag_std']= fft_imag.std()
            data.loc[i,f'{sensor}_fft_imag_max'] = fft_imag.max()
            data.loc[i,f'{sensor}_fft_imag_min'] = fft_imag.min()
            data.loc[i,f'{sensor}_fft_imag_median']= np.median(fft_imag)
            data.loc[i,f'{sensor}_fft_imag_skew'] = spstats.skew(fft_imag)
            data.loc[i,f'{sensor}_fft_imag_kurtosis']= spstats.kurtosis(fft_imag)

            #roll = f.rolling(300)
            #roll_300_mean = roll.mean()
            #roll_300_max = roll.max()
            #roll_300_min = roll.min()
            #roll_300_dist = roll_300_max - roll_300_min
            #roll_300_dist_diff =roll_300_dist.diff()

            #data.loc[i,f'{sensor}_roll_300_dist_min']=roll_300_dist.min(axis=0)
            #data.loc[i,f'{sensor}_roll_300_dist_max']=roll_300_dist.max(axis=0)
            #data.loc[i,f'{sensor}_roll_300_dist_diff_max']=roll_300_dist_diff.min(axis=0)
            #data.loc[i,f'{sensor}_roll_300_dist_diff_min']=roll_300_dist_diff.min(axis=0)
            #data.loc[i,f'{sensor}_roll_300_mean_min']=roll_300_dist.min(axis=0)
            #data.loc[i,f'{sensor}_roll_300_mean_max']=roll_300_dist.max(axis=0)


            #roll = f.rolling(3000)
            #roll_3000_mean = roll.mean()
            #roll_3000_max = roll.max()
            #roll_3000_min = roll.min()
            #roll_3000_dist = roll_3000_max - roll_3000_min
            #roll_3000_dist_diff = roll_3000_dist.diff()

            #data.loc[i,f'{sensor}_roll_3000_dist_max']=roll_3000_dist.min(axis=0)
            #data.loc[i,f'{sensor}_roll_3000_dist_min']=roll_3000_dist.max(axis=0)
            #data.loc[i,f'{sensor}_roll_3000_dist_diff_max']=roll_3000_dist_diff.min(axis=0)
            #data.loc[i,f'{sensor}_roll_3000_dist_diff_min']=roll_3000_dist_diff.min(axis=0)
            #data.loc[i,f'{sensor}_roll_3000_mean_min']=roll_300_mean.min(axis=0)
            #data.loc[i,f'{sensor}_roll_3000_mean_max']=roll_300_mean.max(axis=0)
        i+=1
    data.fillna(0)
    return data

In [ ]:
train_data=pd.read_csv('/kaggle/input/volcaniceruptiondatasets/train_data.csv')
train_data=train_data.iloc[:,1:]
train_data.fillna(0,inplace=True)

In [ ]:
train_data

In [ ]:
X_test=pd.read_csv('/kaggle/input/volcaniceruptiondatasets/test_data.csv')
X_test.drop(["time_to_eruption"],axis=1,inplace=True)
X_test.fillna(0,inplace=True)
X_test=X_test.iloc[:,2:]

In [ ]:
#imputer=SimpleImputer(strategy="mean")
#pd.DataFrame(imputer.fit_transform(train_data.iloc[:,2:]))
#test_data_imp=pd.DataFrame(imputer.fit_transform(test_data))

# Features Scaling

In [ ]:
features = list(train_data.drop(["segment_id", "time_to_eruption"], axis=1).columns)

In [ ]:
scaler=StandardScaler()
scaler.fit(train_data[features])

In [ ]:
train_data[features]=pd.DataFrame(scaler.transform(train_data[features]))
X_test=scaler.transform(X_test)

In [ ]:
X_train=train_data[features]
y_train=train_data["time_to_eruption"]

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_train.shape

In [ ]:
import lightgbm as lgbm
target_name = ["time_to_eruption"]

sub_preds = np.zeros(X_test.shape[0])

model = lgbm.LGBMRegressor(n_estimators=3000,metric="mse",num_leaves=400,random_state=42,max_bins=3000)

model.fit(X_train, y_train, eval_set= [(X_train, y_train)], eval_metric="mae", verbose=12)

sub_preds += model.predict(X_test)

bruh dw ik that i overfitted

:D

In [ ]:
submission = pd.DataFrame()
submission['segment_id'] = test["segment_id"]
submission['time_to_eruption'] = sub_preds
submission.to_csv('submission.csv', header=True, index=False)